# Excercise 1B2

Import modules, ```numpy```, set constants.

In [343]:
from typing import Tuple

import numpy as np

LIGHT_SPEED = 299792.458

## Step 1

Complete first step of solution. 

Needs ```locations``` of the satellites. 
$$
\begin{bmatrix} 
a_1 & b_1 & c_1 \\
a_2 & b_2 & c_2 \\
a_3 & b_3 & c_3 \\
a_4 & b_4 & c_4 \\
\end{bmatrix}
$$

And needs the ```transmission_times``` in a ```numpy``` array.
$\begin{bmatrix} t_1 \\ t_2 \\ t_3 \\ t_4 \end{bmatrix}$

In [344]:
def step_1(
    locations: np.ndarray,
    transmission_times: np.ndarray
) -> Tuple[
    np.ndarray,
    np.ndarray,
    np.ndarray,
    np.ndarray,
    np.ndarray,
]:
    vector_1: np.ndarray = (2 * (locations[1:, 0] - locations[0, 0])).reshape((3, 1))
    vector_2: np.ndarray = (2 * (locations[1:, 1] - locations[0, 1])).reshape((3, 1))
    vector_3: np.ndarray = (2 * (locations[1:, 2] - locations[0, 2])).reshape((3, 1))
    vector_4: np.ndarray = (2 * LIGHT_SPEED**2 * (transmission_times[0] - transmission_times[1:])).reshape((3, 1))
    vector_5: np.ndarray = (
        locations[0, 0] ** 2 - locations[1:, 0] ** 2 + 
        locations[0, 1] ** 2 - locations[1:, 1] ** 2 +
        locations[0, 2] ** 2 - locations[1:, 2] ** 2 + 
        LIGHT_SPEED ** 2 * (transmission_times[1:] ** 2 - transmission_times[0] ** 2)
    ).reshape((3, 1))
    
    return (vector_1, vector_2, vector_3, vector_4, vector_5)

## Step 2

Calculates the second step. Needs vectors calculated in the first step. Names are not entirely correct anymore, but changing it takes too much time.

In [345]:
def step_2(
    vector_2: np.ndarray, 
    vector_3: np.ndarray, 
    vector_4: np.ndarray, 
    vector_5: np.ndarray,
    vector_given: np.ndarray
) -> Tuple[float, float]:
    alpha = (-1 * 
        np.linalg.det(np.concatenate((vector_2, vector_3, vector_5), axis = 1)) / 
        np.linalg.det(np.concatenate((vector_2, vector_3, vector_given), axis = 1))
    )

    beta = (-1 * 
        np.linalg.det(np.concatenate((vector_2, vector_3, vector_4), axis = 1)) / 
        np.linalg.det(np.concatenate((vector_2, vector_3, vector_given), axis = 1))
    )
    return (alpha, beta)

## Step 3

Calculates the third step of the calculation. Needs the $\alpha_x, \; \beta_x, \; \alpha_y, \; \beta_y, \; \alpha_z, \; \beta_z.$

Also needs the position $(a_i, b_i, z_i)$ of a satellite, and the transmission time $t_i$.

In [346]:
def step_3(
    alpha_x: float,
    beta_x: float,
    alpha_y: float,
    beta_y: float, 
    alpha_z: float, 
    beta_z: float,
    position_1_x: float, 
    position_1_y: float, 
    position_1_z: float,
    transmission_time_1: float,
):
    q_0 = (
        position_1_x ** 2 +
        position_1_y ** 2 +
        position_1_z ** 2 -
        (LIGHT_SPEED ** 2) * (transmission_time_1 ** 2) +
        (alpha_x ** 2) +
        (alpha_y ** 2) +
        (alpha_z ** 2)
        -2 * (
            position_1_x * alpha_x +
            position_1_y * alpha_y +
            position_1_z * alpha_z
        ))

    q_1 = (
        -2 * (
            position_1_x * beta_x +
            position_1_y * beta_y +
            position_1_z * beta_z -
            (LIGHT_SPEED ** 2) * transmission_time_1
        ) 
        + 2 * (
            alpha_x * beta_x +
            alpha_y * beta_y +
            alpha_z * beta_z
        ))

    q_2 = (
        (beta_x ** 2) +
        (beta_y ** 2) +
        (beta_z ** 2) -
        (LIGHT_SPEED ** 2)
    )

    return (q_0, q_1, q_2)

## ABC-formula

The best formula that exists in mathematics, if you do not know how to solve quadratic equations using the "kwadraat afsplitsen" method.

In [347]:
def abc_formula(
    a: float,
    b: float,
    c: float,
):
    determinant: float  = np.sqrt((b ** 2) - 4 * a * c) #type:ignore
    value_1 = ((-1 * b - determinant) / (2 * a))
    value_2 = ((-1 * b + determinant) / (2 * a))
    return (value_1, value_2)

## Calculate location

Calculates the location, using the variables given.

In [348]:
def calculate_location(
    locations: np.ndarray,
    transmission_times: np.ndarray
):
    vector_1, vector_2, vector_3, vector_4, vector_5 = step_1(locations, transmission_times)
    
    alpha_x, beta_x = step_2(vector_2, vector_3, vector_4, vector_5, vector_1)
    alpha_y, beta_y = step_2(vector_1, vector_3, vector_4, vector_5, vector_2)
    alpha_z, beta_z = step_2(vector_1, vector_2, vector_4, vector_5, vector_3)

    position_1_x = locations[0, 0]
    position_1_y = locations[0, 1]
    position_1_z = locations[0, 2]
    transmission_time_1 = transmission_times[0]
    
    q_0, q_1, q_2 = step_3(alpha_x, beta_x, alpha_y, beta_y, alpha_z, beta_z, position_1_x, position_1_y, position_1_z, transmission_time_1)
    time_difference_1, time_difference_2 = abc_formula(q_2, q_1, q_0)

    location_x_1 = alpha_x + beta_x * time_difference_1
    location_y_1 = alpha_y + beta_y * time_difference_1
    location_z_1 = alpha_z + beta_z * time_difference_1

    location_x_2 = alpha_x + beta_x * time_difference_2
    location_y_2 = alpha_y + beta_y * time_difference_2
    location_z_2 = alpha_z + beta_z * time_difference_2

    print(f"{location_x_1=}, {location_y_1=}, {location_z_1=}")
    print(f"{location_x_2=}, {location_y_2=}, {location_z_2=}")

    return (
        (location_x_1, location_y_1, location_z_1), 
        (location_x_2, location_y_2, location_z_2)
    )

## Main

The main program.

In [349]:
def main():
    locations = np.array([
        [15600, 7540, 20140], 
        [18760, 2750, 18610], 
        [17610, 14630, 13480], 
        [19170, 610, 18390]
    ])

    transmission_times = np.array([
        0.07074,
        0.07220,
        0.07690,
        0.07242
    ])

    calculate_location(locations, transmission_times)

main()

location_x_1=-39.74783734818158, location_y_1=-134.27414436066428, location_z_1=-9413.624553735734
location_x_2=-41.77270957081522, location_y_2=-16.78919410652558, location_z_2=6370.059559223332
